In [111]:
import pandas as pd
import requests
import time
import json
dtype={
'COD_RH':str,
'NRO_ID_GRUPO':str,
'COD_RH_MUNICIPIO':str,
'COD_INST':str,
'COD_INST_AVALA':str,
'COD_INST_MACRO':str,
'COD_INSTITUCION':str,         
'COD_INST_OTRO':str,
'ID_DEPARTAMENTO':str,
'COD_DEPA_DANE': str,
'COD_MUNI_DANE':str,
'COD_RH_OTRO':str,
'COD_RH_COAUTOR':str,
'COD_RH_CREA':str
 }

In [112]:
#cp product_type.json product_type_bak.json

In [237]:
f=open('product_type.json','r')
df=json.load(f)
f.close()

In [238]:
len(df)

43204

In [217]:
ll=[]
for i in range(len(df)):
    d=df[i]
    dd={}
    dd['SGL_CATEGORIA']=d.get('SGL_CATEGORIA')
    while d.get('product_type'):
        if i%10000==0:
            print(i,end='\r')
        #print(d.get('product_type'))
        d=d.get('product_type')[0]
        dd[f'NIVEL_{d.get("NRO_NIVEL")}']=f'{d.get("COD_TIPO_PRODUCTO")}: {d.get("TXT_NME_TIPO_PRODUCTO")}'
    if dd:
        ll.append(dd)

In [218]:
pf=pd.DataFrame(ll)

In [219]:
pf=pf[['SGL_CATEGORIA',  'NIVEL_0', 'NIVEL_1', 'NIVEL_2', 'NIVEL_3']]

In [220]:
len(pf['NIVEL_0'].unique())

8

In [221]:
len(pf['NIVEL_1'].unique())

53

In [222]:
len(pf['NIVEL_2'].unique())

121

In [223]:
len(pf['NIVEL_3'].unique())

2

In [224]:
len(pf['SGL_CATEGORIA'].unique())

74

In [225]:
pf['NIVEL_3'].dropna().shape

(10,)

In [226]:
def get_nivel_max(row,labels=['Indefinido', 'Otro','Otra']):
    for l in list(range(4))[::-1]:
        if row.get(f'NIVEL_{l}').split(': ')[-1].strip() not in labels:
            NIVEL_MAX=row.get(f'NIVEL_{l}')
            break
    return NIVEL_MAX

In [227]:
pf['NIVEL_MAX']=pf.fillna('Indefinido').apply(lambda row: get_nivel_max(row),axis='columns')

Fix `SGL_CATEGORIA`

In [232]:
pfu=pf.fillna('').drop_duplicates().reset_index(drop=True)

In [234]:
pfu['CATEGORIAS']=pfu['NIVEL_MAX'].apply(lambda s: 
   pfu[pfu.fillna(False)['NIVEL_MAX']==s]['SGL_CATEGORIA'].dropna().str.split('_').str[0].unique()                      
                     )

In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth',200)

In [236]:
pfu.drop_duplicates(subset='NIVEL_MAX').fillna('').sort_values(['NIVEL_0','NIVEL_1','NIVEL_2']).reset_index(drop=True)

,SGL_CATEGORIA,NIVEL_0,NIVEL_1,NIVEL_2,NIVEL_3,NIVEL_MAX,CATEGORIAS
0,,1: Producción bibliográfica,11: Artículo,111: Publicado en revista especializada,,111: Publicado en revista especializada,"[, ART-ART, ART-GC, ART-00, PID-00, PF-EX, PID-PID, PF-00, PF-PF, PE-PE, PE-00]"
1,,1: Producción bibliográfica,11: Artículo,112: Corto (Resumen),,112: Corto (Resumen),"[, ART-00, ART-ART, ART-GC, PID-00, PID-PID, PE-PE, PF-00]"
2,ART-ART_D,1: Producción bibliográfica,11: Artículo,113: Revisión (Survey),,113: Revisión (Survey),"[ART-ART, , ART-00, ART-GC, PID-PID, PID-00, PE-PE]"
3,ART-ART_A1,1: Producción bibliográfica,11: Artículo,114: Caso clínico,,114: Caso clínico,"[ART-ART, , ART-GC, PID-00, ART-00]"
4,,1: Producción bibliográfica,12: Trabajos en eventos (Capítulos de memoria),121: Completo,,121: Completo,[]
5,,1: Producción bibliográfica,12: Trabajos en eventos (Capítulos de memoria),122: Resumen,,122: Resumen,[]
6,,1: Producción bibliográfica,13: Libro,131: Otro libro publicado,,131: Otro libro publicado,"[, PID-00, PE-PE]"
7,,1: Producción bibliográfica,13: Libro,132: Capítulo de libro,,132: Capítulo de libro,"[, PID-PID, PID-00, PF-00, PF-PF]"
8,,1: Producción bibliográfica,13: Libro,133: Libro pedagógico y/o de divulgación,,133: Libro pedagógico y/o de divulgación,"[, PF-00, PID-00, PE-PE]"
9,,1: Producción bibliográfica,13: Libro,134: Libro resultado de investigación,,134: Libro resultado de investigación,"[, PID-00, PID-PID, PE-PE, PF-00]"
